# Neo4j Population and Queries for TreeHouse

This notebook is designed to populate a Neo4j graph database with family member nodes and their relationships. It also includes a set of queries to interact with the graph, ensuring that the data has been correctly imported and enabling meaningful analysis of family relationships.

## Key Components
1. **Data Import**: Create family member nodes and their relationships.
2. **Validation Queries**: Ensure that the data has been correctly imported into the database.
3. **Project Queries**: Perform various operations such as finding descendants, siblings, extended family members, and more.

Let’s get started!

## Data Import: Adding Nodes and Relationships

In this section, we populate the Neo4j database with nodes representing family members and relationships such as `PARENT`, `SIBLING`, and `SPOUSE`. These nodes and relationships form the foundation for all queries and analyses.

### Libraries
1. **`py2neo`**: Used for managing nodes and relationships.
2. **`neo4j`**: Provides session-based interaction with the Neo4j database.
3. **`json` and `pandas`**: Handle and preview JSON data.

In [1]:
!pip install py2neo
from py2neo import Graph, Node, Relationship
import neo4j
import json
import pandas as pd


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\anani\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Connect to Neo4j and Clean Database

This section establishes a connection to the Neo4j database and ensures the database is empty before importing new data.
- **Connection**: Connects to Neo4j using credentials.
- **Cleanup**: Deletes all nodes and relationships for a clean slate.


In [2]:
def connect_db():
    driver = neo4j.GraphDatabase.driver(uri="neo4j://localhost:7687", auth=("neo4j","password"))
    session = driver.session(database="neo4j")
    return session
    
def wipe_out_db(session):
    # wipe out database by deleting all nodes and relationships
    
    # similar to SELECT * FROM graph_db in SQL
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

session = connect_db()
wipe_out_db(session) 

## Load and Preview Family Data

The family data is loaded from a JSON file and displayed in a table for verification.
- **Attributes**:
  - Name
  - Birth date
  - Gender
  - Relationships


In [3]:
# Load family member data from JSON file
with open("family_members.json", "r") as file:
    family_data = json.load(file)

# Preview the data
pd.DataFrame(family_data).head()

_id          name  birth_date      gender  \
0  Family1_Member001   Jacob Lewis  1985-12-13  Non-binary   
1  Family1_Member002   Noah Wright  1971-09-13        Male   
2  Family1_Member003  Oliver Lewis  2004-06-10        Male   
3  Family1_Member004  Harper Lewis  1974-01-17      Female   
4  Family1_Member005     Zoe Lewis  1984-08-08  Non-binary   

                                          milestones  \
0  [{'type': 'Birthday', 'date': '1985-12-13'}, {...   
1  [{'type': 'Birthday', 'date': '1971-09-13'}, {...   
2       [{'type': 'Birthday', 'date': '2004-06-10'}]   
3  [{'type': 'Birthday', 'date': '1974-01-17'}, {...   
4  [{'type': 'Birthday', 'date': '1984-08-08'}, {...   

                                       relationships  \
0  [{'type': 'parent', 'related_to': 'Family1_Mem...   
1  [{'type': 'parent', 'related_to': 'Family1_Mem...   
2  [{'type': 'child', 'related_to': 'Family1_Memb...   
3  [{'type': 'parent', 'related_to': 'Family1_Mem...   
4  [{'type': 'parent', 'related_to': 'Family1_Mem...   

                              interests      occupation favorite_color  \
0  [Blogging, 3D Printing, Calligraphy]       Scientist          Peach   
1             [Yoga, Mixology, Running]       Geologist         Silver   
2       [Cycling, Horse Riding, Gaming]         Athlete           Plum   
3                     [Skiing, Cooking]  Police Officer          Olive   
4     [Cooking, Mixology, Horse Riding]         Teacher         Silver   

   has_pet family_id  
0    False   Family1  
1     True   Family1  
2    False   Family1  
3     True   Family1  
4     True   Family1

## Create Nodes

This section creates nodes for each family member in the Neo4j database. Each node has the following attributes:
- `name`
- `birth_date`
- `gender`
- `favorite_color` (if available)
- `occupation` (if available)


In [4]:
# Create nodes in Neo4j for each family member
for member in family_data:
    query = '''
    MERGE (p:Person {id: $id})
    SET p.name = $name,
        p.birth_date = $birth_date,
        p.gender = $gender,
        p.favorite_color = $favorite_color,
        p.occupation = $occupation,
        p.notes = $notes
    '''
    session.run(query, 
                id=member["_id"], 
                name=member["name"], 
                birth_date=member["birth_date"], 
                gender=member["gender"], 
                favorite_color=member.get("favorite_color", None), 
                occupation=member.get("occupation", None), 
                notes=member.get("notes", None))
print("Nodes created successfully!")


Nodes created successfully!


## Create Relationships

Relationships between family members are created based on the "relationships" field in the JSON data.


In [5]:
# Create relationships in Neo4j for each family member
for member in family_data:
    for rel in member["relationships"]:
        query = '''
        MATCH (from:Person {id: $from_id}), (to:Person {id: $to_id})
        MERGE (from)-[r:%s]->(to)
        ''' % rel["type"].upper()  # Use the specific relationship type
        session.run(query, from_id=member["_id"], to_id=rel["related_to"])
print("Relationships created successfully!")

Relationships created successfully!


## Validation Queries: Ensuring Accurate Data Import

Once the nodes and relationships are created, we run a set of basic queries to validate the data import. These queries verify:
- The total count of nodes and relationships.
- The types of relationships present in the database.
- A preview of nodes and relationships.

These checks ensure that the database has been populated as expected.


In [6]:
query = '''
    MATCH (n)-[r]->(m)
    RETURN n.id AS start_node, type(r) AS relationship, m.id AS end_node
'''
result = session.run(query)
relationships_df = pd.DataFrame([record.data() for record in result])
print(relationships_df)

            start_node relationship           end_node
0    Family1_Member001      SIBLING  Family1_Member015
1    Family1_Member001       SPOUSE  Family1_Member014
2    Family1_Member001       SPOUSE  Family1_Member009
3    Family1_Member002       PARENT  Family1_Member015
4    Family1_Member002      SIBLING  Family1_Member010
..                 ...          ...                ...
359  Family5_Member014       SPOUSE  Family5_Member008
360  Family5_Member015       PARENT  Family5_Member001
361  Family5_Member015      SIBLING  Family5_Member005
362  Family5_Member015       SPOUSE  Family5_Member013
363  Family1_Member001       PARENT  Family1_Member003

[364 rows x 3 columns]


In [7]:
query = '''
    MATCH (child:Person)-[:PARENT]->(parent:Person)
    RETURN child.id AS child_id, parent.id AS parent_id
'''
result = session.run(query)
parent_child_df = pd.DataFrame([record.data() for record in result])
print(parent_child_df)

             child_id          parent_id
0   Family1_Member001  Family1_Member003
1   Family1_Member002  Family1_Member015
2   Family1_Member003  Family1_Member011
3   Family1_Member004  Family1_Member011
4   Family1_Member005  Family1_Member008
..                ...                ...
70  Family5_Member011  Family5_Member007
71  Family5_Member012  Family5_Member007
72  Family5_Member013  Family5_Member002
73  Family5_Member014  Family5_Member010
74  Family5_Member015  Family5_Member001

[75 rows x 2 columns]


In [8]:
query = '''
    MATCH (person:Person)-[:SPOUSE]-(spouse:Person)
    RETURN person.id AS person_id, spouse.id AS spouse_id
'''
result = session.run(query)
spouse_relationships_df = pd.DataFrame([record.data() for record in result])
print(spouse_relationships_df)

             person_id          spouse_id
0    Family1_Member001  Family1_Member002
1    Family1_Member001  Family1_Member009
2    Family1_Member001  Family1_Member014
3    Family1_Member002  Family1_Member009
4    Family1_Member002  Family1_Member008
..                 ...                ...
257  Family5_Member014  Family5_Member005
258  Family5_Member015  Family5_Member013
259  Family5_Member015  Family5_Member014
260  Family5_Member015  Family5_Member009
261  Family5_Member015  Family5_Member008

[262 rows x 2 columns]


In [9]:
query = '''
    MATCH (child:Person)-[:PARENT]->(parent:Person)
    RETURN child.id AS child_id, parent.id AS parent_id
'''
result = session.run(query)
parent_child_df = pd.DataFrame([record.data() for record in result])
print(parent_child_df)

             child_id          parent_id
0   Family1_Member001  Family1_Member003
1   Family1_Member002  Family1_Member015
2   Family1_Member003  Family1_Member011
3   Family1_Member004  Family1_Member011
4   Family1_Member005  Family1_Member008
..                ...                ...
70  Family5_Member011  Family5_Member007
71  Family5_Member012  Family5_Member007
72  Family5_Member013  Family5_Member002
73  Family5_Member014  Family5_Member010
74  Family5_Member015  Family5_Member001

[75 rows x 2 columns]


In [10]:
query = '''
    MATCH (sibling1:Person)-[:PARENT]->(commonParent)<-[:PARENT]-(sibling2:Person)
    WHERE sibling1.id <> sibling2.id
    RETURN sibling1.id AS sibling1_id, sibling1.name AS sibling1_name,
           sibling2.id AS sibling2_id, sibling2.name AS sibling2_name,
           commonParent.id AS parent_id, commonParent.name AS parent_name
    ORDER BY parent_id, sibling1_id, sibling2_id
'''
result = session.run(query)
all_siblings_df = pd.DataFrame([record.data() for record in result])
print(all_siblings_df)

           sibling1_id    sibling1_name        sibling2_id    sibling2_name  \
0    Family1_Member001      Jacob Lewis  Family1_Member007       Luna Lewis   
1    Family1_Member001      Jacob Lewis  Family1_Member011      Sofia Lewis   
2    Family1_Member007       Luna Lewis  Family1_Member001      Jacob Lewis   
3    Family1_Member007       Luna Lewis  Family1_Member011      Sofia Lewis   
4    Family1_Member011      Sofia Lewis  Family1_Member001      Jacob Lewis   
..                 ...              ...                ...              ...   
345  Family5_Member012     Andrew Perez  Family5_Member011     Olivia Perez   
346  Family5_Member008  Benjamin Thomas  Family5_Member014      Sofia Perez   
347  Family5_Member014      Sofia Perez  Family5_Member008  Benjamin Thomas   
348  Family5_Member009   William Rivera  Family5_Member010   Penelope Perez   
349  Family5_Member010   Penelope Perez  Family5_Member009   William Rivera   

             parent_id     parent_name  
0    Famil

## Project Queries: Analyzing Family Relationships

This section contains all the queries required for the project, including:
1. **Find All Descendants**: Retrieves all descendants of a specific family member.
2. **Retrieve Siblings**: Finds siblings of a family member.
3. **Get Spouse**: Retrieves the spouse of a family member.
4. **Find Common Ancestors**: Identifies shared ancestors between two family members.
5. **Retrieve Extended Family**: Queries extended family members such as cousins, aunts, and uncles.
6. **Get Full Family Tree**: Generates a hierarchical tree starting from a specific family member.
7. **Add Parent-Child Relationship**: Creates a new parent-child relationship dynamically.

### 1. Find All Descendants

This query retrieves all descendants of a specified family member by traversing the `PARENT` relationships.

In [11]:
query = '''
    MATCH (p:Person {name: $name})-[:PARENT*]->(descendant:Person)
    RETURN DISTINCT descendant.name AS name, descendant.gender AS gender
'''
result = session.run(query, name="Amelia Perez")
descendants_df = pd.DataFrame([record.data() for record in result])
print(descendants_df)


            name  gender
0      Ava Perez  Female
1   Daniel Perez    Male
2   Olivia Perez  Female
3     Luna Perez    Male
4     Liam Perez  Female
5    James Perez  Female
6  Abigail Perez    Male


### 2. Retrieve Siblings

This query retrieves all siblings of a specified family member using the `SIBLING` relationship.


In [12]:
query = '''
    MATCH (p:Person {name: $name})-[:SIBLING]-(sibling:Person)
    RETURN DISTINCT sibling.name AS name, sibling.gender AS gender
'''
result = session.run(query, name="Amelia Perez")
siblings_df = pd.DataFrame([record.data() for record in result])
print(siblings_df)


            name gender
0  Abigail Perez   Male


### 3. Get Spouse

This query retrieves the spouse of a specified family member using the `SPOUSE` relationship.


In [13]:
query = '''
    MATCH (p:Person {name: $name})-[:SPOUSE]-(spouse:Person)
    RETURN DISTINCT spouse.name AS name, spouse.gender AS gender
'''
result = session.run(query, name="Isabella King")
spouse_df = pd.DataFrame([record.data() for record in result])
print(spouse_df)


              name      gender
0  Samuel Phillips  Non-binary
1     Michael King        Male
2    Andrew Wright  Non-binary
3      Robert King        Male
4       Lucas King      Female
5     William King  Non-binary


### 4. Find Common Ancestors

This query identifies shared ancestors between two family members by traversing the `PARENT` relationships.


In [14]:
query = '''
    MATCH (a:Person {name: $name1})-[:PARENT*]->(common_ancestor:Person)<-[:PARENT*]-(b:Person {name: $name2})
    RETURN DISTINCT common_ancestor.name AS name, common_ancestor.gender AS gender
'''
result = session.run(query, name1="Amelia Perez", name2="Andrew Perez")
common_ancestors_df = pd.DataFrame([record.data() for record in result])
print(common_ancestors_df)


         name  gender
0  Luna Perez    Male
1  Liam Perez  Female


### 5. Retrieve Extended Family

This query retrieves extended family members such as cousins, aunts, and uncles within two levels of relationships.


In [15]:
query = '''
    MATCH (p:Person {name: $name})-[:PARENT*1..2]-(extended_family:Person)
    WHERE NOT (p)-[:PARENT|SIBLING|SPOUSE]-(extended_family)
    RETURN DISTINCT extended_family.name AS name, extended_family.gender AS gender
'''
result = session.run(query, name="Anna Rogers")
extended_family_df = pd.DataFrame([record.data() for record in result])
print(extended_family_df)


          name      gender
0  Noah Wright        Male
1   Luke Moore  Non-binary


### 6. Get Full Family Tree

This query generates a hierarchical tree starting from a specific family member, displaying all descendants along with their relationship types.

In [16]:
query = '''
    MATCH (root:Person {name: $name})-[rel]->(descendant:Person)
    RETURN root.name AS root, type(rel) AS relationship, descendant.name AS descendant
'''
result = session.run(query, name="Amelia Perez")
family_tree_with_relationships_df = pd.DataFrame([record.data() for record in result])
print(family_tree_with_relationships_df)


           root relationship     descendant
0  Amelia Perez      SIBLING  Abigail Perez
1  Amelia Perez       PARENT      Ava Perez
2  Amelia Perez       SPOUSE   Andrew Perez
3  Amelia Perez       SPOUSE    Sofia Perez


### 7. Add Parent-Child Relationship

This query creates a `PARENT` relationship between two family members. This is useful for adding new family connections dynamically.


In [17]:
query = '''
    MATCH (parent:Person {name: $parent_name}), (child:Person {name: $child_name})
    MERGE (parent)-[:PARENT]->(child)
'''
session.run(query, parent_name="Amelia Perez", child_name="Andrew Perez")
print("Parent-child relationship added successfully!")


Parent-child relationship added successfully!
